##### There are two folders for quite similar contents yet different years. Each folders has almost differet function since it's different contents. I read all files from the target folder in order to extract the wanted variables. There are two folders; BalanceSheet and Instantement. Both has files with different variables for same companies. There are duplication in code for some variables are exsited in speical form and it need to a trick in the code to find them

In [1]:
#Import all wanted libraries
import pandas as pd
from tabula import read_pdf
from tabulate import tabulate
import PyPDF2
import os
import re
import camelot
import time 


### 2021 Folders

In [527]:
# Define the start time to know the total time to run 
start_time=time.time()
# Define two lists to see which files are broken and cannot be read.
correct_file=[]
wrong_file=[]
#Define the main Dataframe columns
columns= ['Stand2020','Cons2020','Stand2021','Cons2021']
profit_data= pd.DataFrame(columns=columns)
# Define the directory of files
folder_path='oman 2021\IncomeInstatement'
# Pick up all files with pdf extenstion.
file_list= [file for file in os.listdir(folder_path) if file.endswith('.pdf')]

for file_name in file_list:
    # combine the file name with path
    pdf_path = os.path.join(folder_path, file_name)

    # Use read_pdf to extract tables from the PDF

    tables = read_pdf(pdf_path,multiple_tables=True,columns=[200,300,300,300],pages='all')
    
    result=check_files(tables,file_name)
    
    if result:
        extract_profit(tables)
        
    else:
        continue
 
 print('Time to Run is:',round((time.time()-start_time)/60), 'Mins.')

Time to Run is: 3 Mins.


In [526]:
# Check the pdf file 

def check_files(tables,file_name):
    global correct_file
    global wrong_file
    if len(tables)>0:
        correct_file.append(file_name)
        result= True
    else: # file return empty tables
        wrong_file.append(file_name)
        result=False
    
    return result   

In [525]:
# Function to extract profits data from files
def extract_profit(tables):
    global profit_data
    global file_name
    # choose the first tables for it has the wanted variable
    table=tables[0]
    # Drop null rows from last columns(as reverse in Arabic tables)
    last_col=table.columns[-1]
    table.dropna(subset=[last_col],inplace=True)
    # list all possibility of the variable, and how it mentioned
    keywords=['الربح/)الخسارة( للفترة','ربح )خسارة( خلال السنة','صافي الربح )الخسارة(','صافي الربح/ )الخسارة( للفترة']
    # Take a loop to look for specific values by order 
    for keyword in keywords:
        matched_row=table.loc[table[last_col]==keyword]
        if not matched_row.empty:
            matched_row=matched_row.iloc[[0]]
            
            break
            
    matched_row=matched_row.iloc[:,:-1]
    matched_row=matched_row.dropna(axis=1)
    # Below condition for some files has the 2 columns and more. 
    if len(matched_row.columns)>2:
        matched_row.columns=profit_data.columns
        
    else:
        matched_row.columns=['Cons2020','Cons2021']
        
    # Set the company name as index 
    matched_row.index=[file_name.split('_')[1]]
    
    profit_data=pd.concat([profit_data,matched_row],ignore_index=False).fillna(0)
    


In [531]:
profit_data.head()

,Stand2020,Cons2020,Stand2021,Cons2021
AAIC,67,336,105,(131)
ABOB,0,"23,968",0,"27,606"
AFIC,0,"1,668,760",0,"1,675,511"
AJSS,"2,158,449","2,166,610","2,954,572","2,962,107"
AMAN,0,"1,079,169",0,"2,276,898"


In [26]:
profit_data

,Cons2020,Stand2020,Cons2021,Stand2021
AAIC,67,336,105,(131)
ABOB,"23,968",0,"27,606",0
AFIC,"1,668,760",0,"1,675,511",0
AJSS,"2,158,449","2,166,610","2,954,572","2,962,107"
AMAN,"1,079,169",0,"2,276,898",0
...,...,...,...,...
SUWP,"10,838",0,"11,335",0
TFCI,"2,070",0,"2,543",0
UBAR,"(269,435)","(812,724)","(613,720)","(1,191,537)"
UFCI,"776,041",0,"1,913,741",0


In [375]:
# Row data from the table
tables[0]

,"3,182,644","2,903,191",إجمالي ممتلكات وآلات ومعدات
0,"127,473","123,157",حق استخدام الأصول
1,"3,310,117","3,026,348",إجمالي الموجودات غیر المتداولة
2,NaN,NaN,الموجودات المتداولة
3,"15,905","3,940",المخزون المتداول
4,"60,251","24,912",مدینون تجاریون وأرصدة مدینة اخرى
5,"(25,878)","(19,817)",النقد والأرصدة المصرفیة
6,"50,278","9,035",إجمالي الأصول المتداولة بخلاف الأصول غیر المتد...
7,"50,278","9,035",إجمالي الموجودات المتداولة
8,"3,360,395","3,035,383",إجمالي الموجودات
9,NaN,NaN,المطلوبات وحقوق المساھمین


In [533]:
# Read files in BalanceSheet
start_time=time.time()
# Specify the PDF file path
correct_file_balance=[]
wrong_file_balance=[]
keywords = ['Total_assets', 'Total_current_assets', 'Total_liabilities', 'Total_current_liabilities']
columns= pd.MultiIndex.from_product([['2020','2021'], keywords])
balance_data= pd.DataFrame(columns=columns)
folder_path='oman 2021\BalanceSheet'
file_list= [file for file in os.listdir(folder_path) if file.endswith('.pdf')]

for file_name in file_list:
    pdf_path = os.path.join(folder_path, file_name)

    tables = read_pdf(pdf_path,multiple_tables=True,columns=[200,300,300,300],pages='all')
    # Function to check files accessibility and return a message.
    result=check_files(tables,file_name)
    if result:
        extract_balance(tables)
        
    else:
        continue
print('Total time:',round((time.time()-start_time)/60), 'in Minute') 


filling data went wrong: 2_BACS_الميزانية_العمومية_30122021_53.pdf
filling data went wrong: 2_BACS_الميزانية_العمومية_30122021_53.pdf
filling data went wrong: 2_BACS_الميزانية_العمومية_30122021_53.pdf
filling data went wrong: 2_BACS_الميزانية_العمومية_30122021_53.pdf
filling data went wrong: 2_BACS_الميزانية_العمومية_30122021_53.pdf
filling data went wrong: 2_BACS_الميزانية_العمومية_30122021_53.pdf
filling data went wrong: 2_BACS_الميزانية_العمومية_30122021_53.pdf
filling data went wrong: 2_BACS_الميزانية_العمومية_30122021_53.pdf
filling data went wrong: 2_GECS_الميزانية_العمومية_15032022_39.pdf
filling data went wrong: 2_MGMC_الميزانية_العمومية_14032022_58.pdf
filling data went wrong: 2_NAPI_الميزانية_العمومية_06032022_9.pdf
filling data went wrong: 2_NGCI__د___à_è___د___è_ر__د_____à_ê_à_è_ر_07032022_9.pdf
filling data went wrong: 2_NMWI_الميزانية_العمومية_28022022_53_U.pdf
filling data went wrong: 2_OCHL__د___à_è___د___è_ر__د_____à_ê_à_è_ر_09032022_50.pdf
filling data went wrong: 2_O

In [532]:
def extract_balance(tables):
    global file_name
    global balance_data
    for i in range(len(tables)):
        
        table=tables[i]
        last_col=table.columns[-1]
        table.dropna(subset=[last_col],inplace=True)
        com_name=file_name.split('_')[1]
        Total_assets=table.loc[table[last_col]=='إجمالي الموجودات']
        Total_liabilities=table.loc[table[last_col]=='إجمالي المطلوبات'] 
        Total_current_liabilities=table.loc[table[last_col]=='إجمالي المطلوبات المتداولة']

        if table.shape[1]==3:
            if not Total_assets.empty:
                balance_data.at[com_name,('2020','Total_assets')]=Total_assets.iloc[0,0]
                balance_data.at[com_name,('2021','Total_assets')]=Total_assets.iloc[0,1]
            if not Total_liabilities.empty and pd.isnull(balance_data.loc[com_name,('2020','Total_liabilities')]):
                balance_data.at[com_name,('2020','Total_liabilities')]=Total_liabilities.iloc[0,0]
                balance_data.at[com_name,('2021','Total_liabilities')]=Total_liabilities.iloc[0,1]
            else:
                continue
            if not Total_current_liabilities.empty and pd.isnull(balance_data.loc[com_name,('2020','Total_current_liabilities')]):
                balance_data.at[com_name,('2020','Total_current_liabilities')]=Total_current_liabilities.iloc[0,0]
                balance_data.at[com_name,('2021','Total_current_liabilities')]=Total_current_liabilities.iloc[0,1]
            else:
                continue

        elif table.shape[1]==4:
            if not Total_assets.empty:
                balance_data.at[com_name,('2020','Total_assets')]=Total_assets.iloc[0,1]
                balance_data.at[com_name,('2021','Total_assets')]=Total_assets.iloc[0,2]
            if not Total_liabilities.empty and pd.isnull(balance_data.loc[com_name,('2020','Total_liabilities')]):
                balance_data.at[com_name,('2020','Total_liabilities')]=Total_liabilities.iloc[0,1]
                balance_data.at[com_name,('2021','Total_liabilities')]=Total_liabilities.iloc[0,2]
            else:
                continue
            if not Total_current_liabilities.empty and pd.isnull(balance_data.loc[com_name,('2020','Total_current_liabilities')]):
                balance_data.at[com_name,('2020','Total_current_liabilities')]=Total_current_liabilities.iloc[0,1]
                balance_data.at[com_name,('2021','Total_current_liabilities')]=Total_current_liabilities.iloc[0,2]
            else:
                continue

        elif table.shape[1]==5:
            if not Total_assets.empty:
                balance_data.at[com_name,('2020','Total_assets')]=Total_assets.iloc[0,1]
                balance_data.at[com_name,('2021','Total_assets')]=Total_assets.iloc[0,3]
            if not Total_liabilities.empty and pd.isnull(balance_data.loc[com_name,('2020','Total_liabilities')]):
                balance_data.at[com_name,('2020','Total_liabilities')]=Total_liabilities.iloc[0,1]
                balance_data.at[com_name,('2021','Total_liabilities')]=Total_liabilities.iloc[0,3]
            else:
                continue
            if not Total_current_liabilities.empty and pd.isnull(balance_data.loc[com_name,('2020','Total_current_liabilities')]):
                balance_data.at[com_name,('2020','Total_current_liabilities')]=Total_current_liabilities.iloc[0,1]
                balance_data.at[com_name,('2021','Total_current_liabilities')]=Total_current_liabilities.iloc[0,3]
            else:
                continue

        else:
            print('filling data went wrong:',file_name)

In [534]:
balance_data.head()

2020                                         \
     Total_assets Total_current_assets Total_liabilities   
AAIC       49,874                  NaN            18,557   
ABOB    2,702,477                  NaN         2,314,127   
AFIC   73,644,427                  NaN        54,174,758   
AINS   80,629,718                  NaN        40,842,828   
AJSS   20,386,131                  NaN         2,109,256   

                                       2021                       \
     Total_current_liabilities Total_assets Total_current_assets   
AAIC                       NaN       47,697                  NaN   
ABOB                       NaN    3,052,556                  NaN   
AFIC                       NaN   71,465,767                  NaN   
AINS                       NaN   75,708,315                  NaN   
AJSS                 1,604,078   18,599,417                  NaN   

                                                  
     Total_liabilities Total_current_liabilities  
AAIC            16,590                       NaN  
ABOB         2,625,559                       NaN  
AFIC        51,090,275                       NaN  
AINS        35,394,276                       NaN  
AJSS         2,427,861                 1,983,029

In [582]:
tables[3]

,Unnamed: 0,Unnamed: 1,Unnamed: 2
0,NaN,"2,000,000",ودائع قصیرة الأجل مصنفة كنقد معادل
1,NaN,"2,000,000",إجمالي النقد المعادل
2,"1,751,544","3,389,624",إجمالي النقد والنقد المعادل
3,"1,751,544","3,389,624",إجمالي النقد والأرصدة المصرفیة
4,NaN,NaN,أصول غیر مالیة أخرى متداولة
5,NaN,NaN,حقوق المساھمین
6,NaN,NaN,الاحتیاطیات
7,NaN,NaN,المطلوبات
8,NaN,NaN,المطلوبات غیر المتداولة
9,NaN,NaN,مخصصات غیر متداولة لمنافع الموظفین


In [ ]:
# Combine both Dataframes
full_data=pd.concat([profit_data,balance_data],axis=1)
full_data.head()

In [550]:
#Export dataframed to excel file
full_data.to_excel('Oman_2020_2021.xlsx')

In [707]:
start_time=time.time()
# Specify the PDF file path
correct_file=[]
wrong_file=[]
columns= ['Cons2020','Cons2021']
profit_data= pd.DataFrame(columns=columns)
folder_path='oman 2021\IncomeInstatement'
file_list= [file for file in os.listdir(folder_path) if file.endswith('.pdf')]

for file_name in file_list:
    pdf_path = os.path.join(folder_path, file_name)
    tables = read_pdf(pdf_path,multiple_tables=True,columns=[200,300,300,300],pages='all')
    result=check_files(tables,file_name)
    if result:
        extract_profit(tables)
        
    else:
        continue 
print('Time to Run is:',round((time.time()-start_time)/60), 'Mins.')

Not Matched in: 3_ABOB_بيانات_الدخل_10032022_6.pdf
Not Matched in: 3_AFIC__ذ_è_د___د_ز__د___»_«___16032022_32.pdf
Not Matched in: 3_AMAN_بيانات_الدخل_24032022_33.pdf
Not Matched in: 3_BKDB__ذ_è_د___د_ز__د___»_«___06032022_42.pdf
Not Matched in: 3_BKMB_بيانات_الدخل_28022022_16.pdf
Not Matched in: 3_BKNZ_بيانات_الدخل_13032022_35.pdf
Not Matched in: 3_BKSB_بيانات_الدخل_15032022_57.pdf
Not Matched in: 3_HBMO__ذ_è_د___د_ز__د___»_«___10032022_35.pdf
Not Matched in: 3_NBOB__ذ_è_د___د_ز__د___»_«___15032022_52.pdf
Not Matched in: 3_OAB__ذ_è_د___د_ز__د___»_«___14032022_19.pdf
Not Matched in: 3_OMRF__ذ_è_د___د_ز__د___»_«___16062022_18.pdf
Not Matched in: 3_ORCI_بيانات_الدخل_13032022_7.pdf
Not Matched in: 3_SHCS__ذ_è_د___د_ز__د___»_«___28022022_15.pdf
Time to Run is: 3 Mins.


In [706]:
def extract_profit(tables):
    global profit_data
    global file_name
    table=tables[0]
    # Drop null rows from last columns(as reverse in Arabic tables)
    last_col=table.columns[-1]
    table.dropna(subset=[last_col],inplace=True)
    com_name=file_name.split('_')[1]

    matched_row=table.loc[table[last_col]=='تكالیف تمویل']

    if not matched_row.empty:
        if matched_row.shape[1]==3:
            profit_data.loc[com_name,'Cons2020']=matched_row.iloc[0,0]
            profit_data.loc[com_name,'Cons2021']=matched_row.iloc[0,1]
        
        elif matched_row.shape[1]==5:
            profit_data.loc[com_name,'Cons2020']=matched_row.iloc[0,1]
            profit_data.loc[com_name,'Cons2021']=matched_row.iloc[0,3]

    else:
        print('Not Matched in:', file_name)
        

In [709]:
# See how much the extracted data got. 
profit_data.shape

(57, 2)

In [717]:
#Set columns names for extracted dataframed
takleef_data=profit_data
takleef_data.columns=['تكاليف2020','تكاليف2021']
takleef_data.head()

,تكاليف2020,تكاليف2021
AAIC,864,263
AJSS,"5,325","3,385"
AMII,"641,339","312,652"
APBS,"2,947,223","2,274,478"
BAHS,"28,635","33,849"


In [719]:
profit_data.head()

,تكاليف2020,تكاليف2021
AAIC,864,263
AJSS,"5,325","3,385"
AMII,"641,339","312,652"
APBS,"2,947,223","2,274,478"
BAHS,"28,635","33,849"


In [714]:
#Export dataframed to excel
old_data=pd.read_excel('Oman_2020_2021.xlsx')
old_data.head()

,Unnamed: 0,Stand2020,Cons2020,Stand2021,Cons2021,"('2020', 'Total_assets')","('2020', 'Total_current_assets')","('2020', 'Total_liabilities')","('2020', 'Total_current_liabilities')","('2021', 'Total_assets')","('2021', 'Total_current_assets')","('2021', 'Total_liabilities')","('2021', 'Total_current_liabilities')"
0,AAIC,67,336,105,(131),"49,874",NaN,"18,557",NaN,"47,697",NaN,"16,590",NaN
1,ABOB,0,"23,968",0,"27,606","2,702,477",NaN,"2,314,127",NaN,"3,052,556",NaN,"2,625,559",NaN
2,AFIC,0,"1,668,760",0,"1,675,511","73,644,427",NaN,"54,174,758",NaN,"71,465,767",NaN,"51,090,275",NaN
3,AJSS,"2,158,449","2,166,610","2,954,572","2,962,107","20,386,131",NaN,"2,109,256","1,604,078","18,599,417",NaN,"2,427,861","1,983,029"
4,AMAN,0,"1,079,169",0,"2,276,898","20,454,864",NaN,"103,799",NaN,"21,206,736",NaN,"24,773",NaN


In [742]:
fullpd.concat([old_data,profit_data],axis=1)

### 2019 Folders 

In [816]:
def extract_balance(tables):
    global file_name
    global balance_data
    for i in range(len(tables)):
        
        table=tables[i]
        # Drop null rows from last columns(as reverse in Arabic tables)
        last_col=table.columns[-1]
        table.dropna(subset=[last_col],inplace=True)


        com_name=file_name.split('_')[1]
        asset_keywords=['اجمالي الاصول','مجموع الأصول','إجمالي الأصول']
        liability_keywords=['إجمالي الالتزامات','مجموع الالتزامات']
        
        Total_assets=table.loc[table[last_col].isin(asset_keywords)]
        Total_liabilities=table.loc[table[last_col].isin(liability_keywords)] 

        if table.shape[1]==3:
            if not Total_assets.empty:
                balance_data.at[com_name,('2018','Total_assets')]=Total_assets.iloc[0,0]
                balance_data.at[com_name,('2019','Total_assets')]=Total_assets.iloc[0,1]
            if not Total_liabilities.empty and pd.isnull(balance_data.loc[com_name,('2018','Total_liabilities')]):
                balance_data.at[com_name,('2018','Total_liabilities')]=Total_liabilities.iloc[0,0]
                balance_data.at[com_name,('2019','Total_liabilities')]=Total_liabilities.iloc[0,1]
                break
            else:
                continue

        elif table.shape[1]==4:
            if not Total_assets.empty:
                balance_data.at[com_name,('2018','Total_assets')]=Total_assets.iloc[0,0]
                balance_data.at[com_name,('2019','Total_assets')]=Total_assets.iloc[0,1]
            if not Total_liabilities.empty and pd.isnull(balance_data.loc[com_name,('2018','Total_liabilities')]):
                balance_data.at[com_name,('2018','Total_liabilities')]=Total_liabilities.iloc[0,0]
                balance_data.at[com_name,('2019','Total_liabilities')]=Total_liabilities.iloc[0,1]
                break
            else:
                continue
          
        elif table.shape[1]==5:
            if not Total_assets.empty:
                balance_data.at[com_name,('2018','Total_assets')]=Total_assets.iloc[0,0]
                balance_data.at[com_name,('2019','Total_assets')]=Total_assets.iloc[0,2]
            if not Total_liabilities.empty and pd.isnull(balance_data.loc[com_name,('2018','Total_liabilities')]):
                balance_data.at[com_name,('2018','Total_liabilities')]=Total_liabilities.iloc[0,0]
                balance_data.at[com_name,('2019','Total_liabilities')]=Total_liabilities.iloc[0,2]
                break
            else:
                continue
           
        else:
            print('filling data went wrong:',file_name)


In [822]:
start_time=time.time()
# Specify the PDF file path
correct_file=[]
wrong_file=[]
keywords = ['Total_assets', 'Total_liabilities']
columns= pd.MultiIndex.from_product([['2018','2019'], keywords])
balance_data= pd.DataFrame(columns=columns)
# profit_data
folder_path='oman 2019\Balancesheet'
file_list= [file for file in os.listdir(folder_path) if file.endswith('.pdf')]
for file_name in file_list:
    pdf_path = os.path.join(folder_path, file_name)

    tables = read_pdf(pdf_path,multiple_tables=True,columns=[500,500,500,500],pages='all')
    
    result=check_files(tables,file_name)
    if result:
        extract_balance(tables)
        
    else:
        continue
print('Time to Run is:',round((time.time()-start_time)/60), 'Mins.')

filling data went wrong: 2_AJSS_الميزانية_العمومية_25002020_22.pdf


Got stderr: Aug 27, 2023 11:36:46 AM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Aug 27, 2023 11:36:46 AM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Aug 27, 2023 11:36:46 AM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Aug 27, 2023 11:36:46 AM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Aug 27, 2023 11:36:46 AM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Aug 27, 2023 11:36:46 AM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Aug 27, 2023 11:36:46 AM org.apache.pdfbox.contentstream.P

filling data went wrong: 2_BKSB_الميزانية_العمومية_12002020_13.pdf
filling data went wrong: 2_GECS_الميزانية_العمومية_08002020_50.pdf


Got stderr: Aug 27, 2023 11:37:23 AM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Aug 27, 2023 11:37:23 AM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Aug 27, 2023 11:37:23 AM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Aug 27, 2023 11:37:23 AM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Aug 27, 2023 11:37:23 AM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Aug 27, 2023 11:37:23 AM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Aug 27, 2023 11:37:23 AM org.apache.pdfbox.contentstream.P

filling data went wrong: 2_HBMO_الميزانية_العمومية_10002020_22.pdf
filling data went wrong: 2_HECI_الميزانية_العمومية_11002020_45.pdf


...

filling data went wrong: 2_TAOI_الميزانية_العمومية_25002020_40.pdf
filling data went wrong: 2_TAOI_الميزانية_العمومية_25002020_40.pdf


Got stderr: Aug 27, 2023 11:40:05 AM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Aug 27, 2023 11:40:05 AM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Aug 27, 2023 11:40:05 AM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Aug 27, 2023 11:40:05 AM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Aug 27, 2023 11:40:05 AM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Aug 27, 2023 11:40:05 AM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Aug 27, 2023 11:40:05 AM org.apache.pdfbox.contentstream.P

filling data went wrong: 2_UBAR_الميزانية_العمومية_26002020_21.pdf


Got stderr: Aug 27, 2023 11:40:09 AM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Aug 27, 2023 11:40:09 AM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Aug 27, 2023 11:40:09 AM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Aug 27, 2023 11:40:09 AM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Aug 27, 2023 11:40:09 AM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Aug 27, 2023 11:40:09 AM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Aug 27, 2023 11:40:09 AM org.apache.pdfbox.contentstream.P

Time to Run is: 4 Mins.
